In [ ]:
# Header starts here.
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
from copy import deepcopy
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    objc = deepcopy(obj)
    try:
        tmp = DX(str(float(objc)))
        objc = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(objc)):
            tmp = DX(str(float(objc[i])))
            objc[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return objc

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm) = ( m/1000, m/100 )
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton
deg    = pi/180

half = S(1)/2

# Header ends here.
#
# https://colab.research.google.com/github/kassbohm/wb-snippets/blob/master/ipynb/HTM_03/Uebungen/3.3_cc.ipynb

from mpmath import radians, degrees, pi

l1, l2 = 3*m/10, 6*m/10
p1 = radians(30)
prec = 0.001

s, p2 = var("s, φ₂")

c1, s1 = cos(p1), sin(p1)
c2, s2 = cos(p2), sin(p2)

xdev = ( l1*c1 - l2*c2 - s ) / m
ydev = ( l1*s1 - l2*s2 ) / m
xdev = xdev.simplify()
ydev = ydev.simplify()

xi = var("xi")
xdev = xdev.subs(s/m, xi)

J11, J12 = diff(xdev, p2), diff(xdev, xi)
J21, J22 = diff(ydev, p2), diff(ydev, xi)

# Jacobian:
J = Matrix([[J11, J12],[J21, J22]])
# pprint(J)

pprint("\n--- Numerical Solution using SymPy's nsolve ---")

# Initial Values:
pprint("\nInitial (φ₂ / deg, ξ):")
p2i, xii = radians(150), 0.8

sol = nsolve( [xdev, ydev], [p2, xi], [p2i, xii], dict=True )
sol = sol[0]
tmp =  Matrix([degrees(sol[p2]), sol[xi]])
pprint("\nSolution:")
pprint("(φ₂ / deg, ξ):")
pprint(iso_round(tmp,prec))

pprint("\n--- Step-by-Step Newton-Solution ---")

pprint("\n--- Step-1 ---")

pprint("\nJacobian in terms of (φ₂, ξ):")
pprint(J)

pprint("\n1: Inital (φ₂, ξ):")
p = Matrix([p2i, xii])
pprint(iso_round(p, prec))
pprint("Note: φ₂ / deg:")
tmp = degrees(p[0])
pprint(iso_round(tmp, prec))

F = Matrix([xdev, ydev])

sub_list = [
    (p2, p2i),
    (xi, xii),
    ]

pprint("\n2: Jacobian:")
Jn = J.subs(sub_list)
# pprint(Jn)
pprint(iso_round(Jn, prec))

pprint("\n3: Deviation Phi:")
Fn = F.subs(sub_list)
pprint(iso_round(Fn, prec))

# Newton-Step:
pprint("\n4: Increment (δφ₂, ξ):")
dp2, dxi = var("δφ₂, δξ")
dp = Matrix([dp2, dxi])
eq = Eq(Jn*dp, -Fn)
sol = solve(eq,[dp2, dxi], dict=True)
sol = sol[0]
dp = Matrix([sol[dp2], sol[dxi]])
pprint(iso_round(dp, prec))
p += dp
pprint("\n5: At end of iteration step:")
pprint("(φ₂, ξ):")
pprint(iso_round(p, prec))
pprint("Note: φ₂ / deg:")
tmp = degrees(p[0])
pprint(iso_round(tmp, prec))

# ---


pprint("\n--- Step-2 ---")
pprint("\n1: Using (φ₂, ξ) from end of last step.")

sub_list = [
    (p2, p[0]),
    (xi, p[1]),
    ]

pprint("\n2: Jacobian:")
Jn = J.subs(sub_list)
pprint(iso_round(Jn, prec))

pprint("\n3: Deviation Phi:")
Fn = F.subs(sub_list)
pprint(iso_round(Fn, prec))

# Newton-Step:
pprint("\n4: Increment (δφ₂, ξ):")
dp2, dxi = var("δφ₂, δξ")
dp = Matrix([dp2, dxi])
eq = Eq(Jn*dp, -Fn)
sol = solve(eq,[dp2, dxi], dict=True)
sol = sol[0]
dp = Matrix([sol[dp2], sol[dxi]])
pprint(iso_round(dp, prec))
p += dp
pprint("\n5: At end of iteration step:")
pprint("(φ₂, ξ):")
pprint(iso_round(p, prec))
pprint("Note: φ₂ / deg:")
tmp = degrees(p[0])
pprint(iso_round(tmp, prec))

# --- Numerical Solution using SymPy's nsolve ---
#
# Initial (φ₂ / deg, ξ):
#
# Solution:
# (φ₂ / deg, ξ):
# ⎡165.522⎤
# ⎢       ⎥
# ⎣ 0.841 ⎦
#
# --- Step-by-Step Newton-Solution ---
#
# --- Step-1 ---
#
# Jacobian in terms of (φ₂, ξ):
# ⎡ 3⋅sin(φ₂)      ⎤
# ⎢ ─────────    -1⎥
# ⎢     5          ⎥
# ⎢                ⎥
# ⎣-0.6⋅cos(φ₂)  0 ⎦
#
# 1: Inital (φ₂, ξ):
# ⎡2.618⎤
# ⎢     ⎥
# ⎣ 0.8 ⎦
# Note: φ₂ / deg:
# 150.000
#
# 2: Jacobian:
# ⎡0.3   -1.0⎤
# ⎢          ⎥
# ⎣0.52  0.0 ⎦
#
# 3: Deviation Phi:
# ⎡-0.021⎤
# ⎢      ⎥
# ⎣-0.15 ⎦
#
# 4: Increment (δφ₂, ξ):
# ⎡0.289⎤
# ⎢     ⎥
# ⎣0.066⎦
#
# 5: At end of iteration step:
# (φ₂, ξ):
# ⎡2.907⎤
# ⎢     ⎥
# ⎣0.866⎦
# Note: φ₂ / deg:
# 166.540
#
# --- Step-2 ---
#
# 1: Using (φ₂, ξ) from end of last step.
#
# 2: Jacobian:
# ⎡0.14   -1.0⎤
# ⎢           ⎥
# ⎣0.584  0.0 ⎦
#
# 3: Deviation Phi:
# ⎡-0.023⎤
# ⎢      ⎥
# ⎣ 0.01 ⎦
#
# 4: Increment (δφ₂, ξ):
# ⎡-0.018⎤
# ⎢      ⎥
# ⎣-0.025⎦
#
# 5: At end of iteration step:
# (φ₂, ξ):
# ⎡2.889⎤
# ⎢     ⎥
# ⎣0.841⎦
# Note: φ₂ / deg:
# 165.525
